# Create Scientific WebAssembly Pipelines

### Learning Objectives

- Gain experience **building** an **[ITK-Wasm](https://wasm.itk.org)** C++ scientific processing pipeline.
- Learn about the concept of **wasm interface types**.
- Learn how to **add custom processing code** to an ITK-Wasm pipeline.
- Learn how to build, run, and test **generated TypeScript and Python binding packages.**  

## Build a custom ITK-Wasm project



### Why ITK-Wasm?

ITK-Wasm provides a **simple, powerful, and interoperable** tooling to *create C++-based wasm scientific processing pipelines*. This tooling is compatible with wasm and open-source scientific community standards can easily be customized to meet your needs.

Specifically, [ITK-Wasm provides the ability](https://wasm.itk.org/en/latest/introduction/parts.html):

- To **easily and reproducibly build scientific C++ codebases** to wasm with the Emscripten or WASI toolchains
- To build the same C++ code *without any modifications* to **native binaries** for debugging or deployment of native binaries
- To create **CLI tests** that can be run on both native and wasm binaries without modification
- To generate **TypeScript / JavaScript bindings and packages**
- To generate a zero-install web application that provides a **live API demo**
- To generate **Python bindings and packages**
- To **manage testing data** shared across CLI and language binding tests
- To **drive the build and testing process** for these constituent parts

### Create an ITK-Wasm project that provides an N-dimensional median image filter

In this tutorial, we will create an ITK-Wasm project that provides an N-dimensional median image filter.

*Note:* Cells in this notebook are shell commands that run on a Jupyter bash kernel. Follow the tutorial [installation instructions](./README.md) to install the bash kernel and required command line executables.

ITK-Wasm projects are intended to be developed in standalone source code repositories, as subdirectories in source code repositories, or as package in a [JavaScript project monorepo](https://www.robinwieruch.de/javascript-monorepos/).

Example ITK-Wasm projects can be found in the [packages/* directory of the *itk-wasm* repository](https://github.com/InsightSoftwareConsortium/itk-wasm/tree/main/packages).

We will start with a project in the tutorial repository *wasm-median/* directory. This contains the skeleton of our ITK-Wasm project. Work is in progress so this skeleton can be generated interactively with the command:

```sh
pnpm create itk-wasm
```

In [1]:
cd wasm-median/
ls

CMakeLists.txt	 median-image-filter  pnpm-workspace.yaml
environment.yml  package.json	      README.md


The skeleton contains the following files:

### *README.md*

A description of the project for the repository.

In [2]:
cat README.md

# median

Apply a median filter to a 2D or 3D spatial image.


### *CMakeLists.txt*

A top level *CMakeLists.txt* [CMake](https://cmake.org) C++ build system configuration file.

In [4]:
cat CMakeLists.txt

cmake_minimum_required(VERSION 3.16)
project(median LANGUAGES CXX)

set(CMAKE_CXX_STANDARD 17)

find_package(ITK REQUIRED COMPONENTS
  WebAssemblyInterface
)
include(${ITK_USE_FILE})

enable_testing()

# Begin create-itk-wasm added pipelines.
add_subdirectory(median-image-filter)
# End create-itk-wasm added pipelines.


### *median-image-filter/CMakeLists.txt*

A *CMakeLists.txt* CMake configuration file for the `median-image-filter` pipeline. This describes how to build the pipeline and adds a default test that produces the CLI help output.

In [5]:
cat median-image-filter/CMakeLists.txt

add_executable(median-image-filter median-image-filter.cxx)
target_link_libraries(median-image-filter PUBLIC ${ITK_LIBRARIES})

add_test(NAME median-image-filter-help COMMAND median-image-filter --help)


### median-image-filter/median-image-filter.cxx

This is the C++ source code for the WebAssembly module.

The source contains a command line interface for a processing pipeline executable. Command line arguments are parsed with an extension to the  modern C++ argument parser, [CLI11](https://github.com/CLIUtils/CLI11), `itk::wasm::Pipeline`. From this interface, and `itk::wasm` **interface types**, we can:

- Build and run as a native executable on the command line.
- Build and run as a wasm module the command on the command line.
- Generate bindings and packages for programming languages.

In [11]:
cat median-image-filter/median-image-filter.cxx

/*=========================================================================

 *  Copyright NumFOCUS
 *
 *  Licensed under the Apache License, Version 2.0 (the "License");
 *  you may not use this file except in compliance with the License.
 *  You may obtain a copy of the License at
 *
 *         https://www.apache.org/licenses/LICENSE-2.0.txt
 *
 *  Unless required by applicable law or agreed to in writing, software
 *  distributed under the License is distributed on an "AS IS" BASIS,
 *  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 *  See the License for the specific language governing permissions and
 *  limitations under the License.
 *
 *=========================================================================*/

#include "itkPipeline.h"
#include "itkInputImage.h"
#include "itkOutputImage.h"
#include "itkSupportInputImageTypes.h"

template <typename TImage>
int MedianFilter(itk::wasm::Pipeline &pipeline, const TImage *inputImage)
{
  using ImageType = 

### *environment.yml*

An *environment.yml* provides the conda environment configuration to build and test Python packages.

In the future, this may also include packages to support development and testing of bindings and packages for other languages such as Java, C#, and Rust. 

At build time, [micromamba](https://mamba.readthedocs.io/en/latest/user_guide/micromamba.html) is downloaded and a conda environment, a conda environment is created from this configuration, and Python packages are built and tested with this environment's tools.

In [6]:
cat environment.yml

name: median
channels:
  - conda-forge
dependencies:
  - pytest
  - python=3.11
  - pip
  - pip:
    - hatch


### *package.json* and *pnpm-workspace.yaml*

The build and test process is driven by [`pnpm`](https://pnpm.io/), the fast, disk-space efficient package manager.

This process is configured by the *package.json* Node Package Manager (NPM) configuration file and *pnpm-workspace.yml*.

`pnpm` uses the `itk-wasm` pnpm build scripts to build the project in parallel following build stage dependencies. WebAssembly binaries are built before bindings are generated, etc.

In [7]:
cat pnpm-workspace.yaml
cat package.json

packages:
  - 'typescript'
{
  "name": "median-build",
  "version": "0.1.0",
  "private": true,
  "packageManager": "pnpm@8.15.1",
  "description": "Scripts to generate median itk-wasm artifacts.",
  "type": "module",
  "itk-wasm": {
    "package-description": "Apply a median filter to a 2D or 3D spatial image.",
    "typescript-package-name": "@itk-wasm/median",
    "python-package-name": "itkwasm-median",
    "repository": "https://github.com/InsightSoftwareConsortium/itk-wasm"
  },
  "license": "Apache-2.0",
  "scripts": {
    "build": "pnpm build:gen:typescript && pnpm build:gen:python",
    "build:emscripten": "itk-wasm pnpm-script build:emscripten",
    "build:emscripten:debug": "itk-wasm pnpm-script build:emscripten:debug",
    "build:wasi": "itk-wasm pnpm-script build:wasi",
    "build:wasi:debug": "itk-wasm pnpm-script build:wasi:debug",
    "build:python:wasi": "itk-wasm pnpm-script build:python:wasi",
    "bindgen:typescript": "itk-wasm pnpm-script bindgen:typescript",
    "

### Build the skeleton

To install the dependencies specified in the *package.json* file, run `pnpm install`.

In [8]:
pnpm install

Progress: resolved 1, reused 0, downloaded 0, added 0
Progress: resolved 3, reused 2, downloaded 0, added 0
Progress: resolved 35, reused 34, downloaded 0, added 0
Progress: resolved 90, reused 88, downloaded 1, added 0
Progress: resolved 93, reused 91, downloaded 2, added 0
Progress: resolved 116, reused 113, downloaded 2, added 0
Progress: resolved 130, reused 125, downloaded 3, added 0
Progress: resolved 134, reused 129, downloaded 5, added 0
Progress: resolved 169, reused 158, downloaded 9, added 0
Progress: resolved 173, reused 162, downloaded 11, added 0
Progress: resolved 197, reused 184, downloaded 13, added 0
Progress: resolved 200, reused 186, downloaded 13, added 0
 WARN  1 deprecated subdependencies found: uuid@3.4.0
Progress: resolved 200, reused 186, downloaded 13, added 0
Packages: +201
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Progress: resolved 200, reused 186, downloaded 13, added 0
Progress: resolved 201, reused 187, downloaded 14

Run the `pnpm build` target.

This runs `build` script, building Emscripten and WASI wasm modules, TypeScript and Python bindings, and Typescript and Python package configurations.

In [14]:
pnpm build


> median-build@0.1.0 build /home/matt/src/ScientificImageAnalysisVisualizationAndArtificialIntelligenceCourse/wasm-median
> pnpm build:gen:typescript && pnpm build:gen:python


> median-build@0.1.0 build:gen:typescript /home/matt/src/ScientificImageAnalysisVisualizationAndArtificialIntelligenceCourse/wasm-median
> itk-wasm pnpm-script build:gen:typescript

>> pnpm -c exec pnpm build:emscripten && pnpm bindgen:typescript && pnpm install && pnpm -r build

> median-build@0.1.0 build:emscripten /home/matt/src/ScientificImageAnalysisVisualizationAndArtificialIntelligenceCourse/wasm-median
> itk-wasm pnpm-script build:emscripten

>> pnpm -c exec itk-wasm build -i itkwasm/emscripten:20231217-40780708
Not searching for unused variables given on the command line.
-- Configuring done
-- Generating done
-- Build files have been written to: /work/emscripten-build
ninja: Entering directory `emscripten-build'
[2/2] Linking CXX executable median-image-filter/median-image-filter.jscxx.o
cache:INFO: g

The output includes *emscripten-build/* and *wasi-build/* directories, which contain the Emscripten and WASI builds, respectively.

The *typescript/* and *python/* directories contain language bindings and package configurations.

In [19]:
ls -ltr

total 128
drwxrwxr-x 2 matt matt  4096 Feb 13 11:45 median-image-filter
-rw-rw-r-- 1 matt matt    61 Feb 13 11:51 README.md
-rw-rw-r-- 1 matt matt    27 Feb 13 11:51 pnpm-workspace.yaml
-rw-rw-r-- 1 matt matt  2643 Feb 13 11:51 package.json
-rw-rw-r-- 1 matt matt   109 Feb 13 11:51 environment.yml
-rw-rw-r-- 1 matt matt 76836 Feb 13 12:15 pnpm-lock.yaml
-rw-rw-r-- 1 matt matt   321 Feb 13 12:32 CMakeLists.txt
drwxrwxr-x 5 matt matt  4096 Feb 13 12:33 python
drwxrwxr-x 4 matt matt  4096 Feb 13 12:33 micromamba
drwxrwxr-x 5 matt matt  4096 Feb 13 12:34 emscripten-build
drwxrwxr-x 6 matt matt  4096 Feb 13 12:34 node_modules
drwxrwxr-x 8 matt matt  4096 Feb 13 12:34 typescript
drwxrwxr-x 3 matt matt  4096 Feb 13 12:34 test
drwxrwxr-x 6 matt matt  4096 Feb 13 12:39 wasi-build


To list the available targets for `pnpm`, run `pnpm run`.

In [15]:
pnpm run

Lifecycle scripts:
  test
    pnpm test:data:download && pnpm build:gen:python && pnpm test:python

Commands available via "pnpm run":
  build
    pnpm build:gen:typescript && pnpm build:gen:python
  build:emscripten
    itk-wasm pnpm-script build:emscripten
  build:emscripten:debug
    itk-wasm pnpm-script build:emscripten:debug
  build:wasi
    itk-wasm pnpm-script build:wasi
  build:wasi:debug
    itk-wasm pnpm-script build:wasi:debug
  build:python:wasi
    itk-wasm pnpm-script build:python:wasi
  bindgen:typescript
    itk-wasm pnpm-script bindgen:typescript
  bindgen:python
    itk-wasm pnpm-script bindgen:python
  build:gen:typescript
    itk-wasm pnpm-script build:gen:typescript
  build:gen:python
    itk-wasm pnpm-script build:gen:python
  build:micromamba
    itk-wasm pnpm-script build:micromamba
  build:python:versionSync
    itk-wasm pnpm-script build:python:versionSync
  publish:python
    itk-wasm pnpm-script publish:python
  test:data:download
    dam download test/data 

To run the CLI help test, run `pnpm test:wasi`.

In [17]:
pnpm test:wasi


> median-build@0.1.0 test:wasi /home/matt/src/ScientificImageAnalysisVisualizationAndArtificialIntelligenceCourse/wasm-median
> itk-wasm pnpm-script test:wasi

>> pnpm -c exec pnpm test:data:download && itk-wasm test -- --output-on-failure

> median-build@0.1.0 test:data:download /home/matt/src/ScientificImageAnalysisVisualizationAndArtificialIntelligenceCourse/wasm-median
> dam download test/data test/data.tar.gz bafkreigpkk3pqcoqzjzcauogw6dml52yig3ksmcrobau5pkoictymizzri https://github.com/InsightSoftwareConsortium/itk-wasm/releases/download/itk-wasm-v1.0.0-b.163/create-itk-wasm-test-data.tar.gz https://bafybeiczuxeuma5cjuli5mtapqnjqypeaum5ikd45zcmfhtt2emp365tca.ipfs.w3s.link/ipfs/bafybeiczuxeuma5cjuli5mtapqnjqypeaum5ikd45zcmfhtt2emp365tca/create-itk-wasm-test-data.tar.gz https://ipfs.filebase.io/ipfs/QmcxyvUKnaoTTwUqEPXwp1sdcbrFh3XnnwckLKVRpctJx9

Internal ctest changing into directory: /work/wasi-build
Test project /work/wasi-build
    Start 1: median-image-filter-help
1/1 Test #1

This runs the equivalent of:

In [20]:
wasmtime run --dir=./ wasi-build/median-image-filter/median-image-filter.wasi.wasm -- --help

         Wasmtime versions -- see this online issue for more information:
         https://github.com/bytecodealliance/wasmtime/issues/7384

         Wasmtime will now execute with the old (<= Wasmtime 13) CLI parsing,
         however this behavior can also be temporarily configured with an
         environment variable:

         - WASMTIME_NEW_CLI=0 to indicate old semantics are desired and silence this warning, or
         - WASMTIME_NEW_CLI=1 to indicate new semantics are desired and use the latest behavior
       Welcome to
  __/\\\\\\\\\\\__/\\\\\\\\\\\\\\\__/\\\________/\\\_
  _\/////\\\///__\///////\\\/////__\/\\\_____/\\\//__
  _____\/\\\___________\/\\\_______\/\\\__/\\\//_____
    _____\/\\\___________\/\\\_______\/\\\\\\//\\\_____
    _____\/\\\___________\/\\\_______\/\\\//_\//\\\____
      _____\/\\\___________\/\\\_______\/\\\____\//\\\___
      _____\/\\\___________\/\\\_______\/\\\_____\//\\\__
        __/\\\\\\\\\\\_______\/\\\_______\/\\\______\//\\\_
        _\////

Add the following C++ code to *median-image-filter.cxx* to provide the logic and implementation of the processing pipeline.

```cpp
  #include "itkMedianImageFilter.h"

  // [...]

  using SmoothingFilterType = itk::MedianImageFilter<ImageType, ImageType>;
  auto smoother = SmoothingFilterType::New();
  smoother->SetInput(inputImage);
  smoother->SetRadius(radius);

  smoother->UpdateLargestPossibleRegion();
  outputImage.Set(smoother->GetOutput());
```

and `ITKSmoothing` to the `ITK` package components in the top level *CMakeLists.txt*.

Re-build.

In [23]:
pnpm build


> median-build@0.1.0 build /home/matt/src/ScientificImageAnalysisVisualizationAndArtificialIntelligenceCourse/wasm-median
> pnpm build:gen:typescript && pnpm build:gen:python


> median-build@0.1.0 build:gen:typescript /home/matt/src/ScientificImageAnalysisVisualizationAndArtificialIntelligenceCourse/wasm-median
> itk-wasm pnpm-script build:gen:typescript

>> pnpm -c exec pnpm build:emscripten && pnpm bindgen:typescript && pnpm install && pnpm -r build

> median-build@0.1.0 build:emscripten /home/matt/src/ScientificImageAnalysisVisualizationAndArtificialIntelligenceCourse/wasm-median
> itk-wasm pnpm-script build:emscripten

>> pnpm -c exec itk-wasm build -i itkwasm/emscripten:20231217-40780708
Not searching for unused variables given on the command line.
-- Configuring done
-- Generating done
-- Build files have been written to: /work/emscripten-build
ninja: Entering directory `emscripten-build'
[2/2] Linking CXX executable median-image-filter/median-image-filter.jscxx.o
cache:INFO: g

**Congratulations!**. You created a WebAssembly module that con median-filter a 2D or 3D scientific image.

## Exercises

### 1. Run the web demo-application

A demo JavaScript application. Build and run it:

```sh
cd typescript
pnpm install
pnpm build
pnpm start
```

### 2. Add a CTest test

Add a CTest test in *median-image-filter/CMakeLists.txt* that runs on the *data/visible-male.nrrd* image.

### 3. Create a Node.js test

Create a Node.js test with the `ava` test runner.

```sh
cd typescript
pnpm install -D ava
# Add the tests in test/node/visible-male.js
```

*Hint:* See the itk-wasm packages for example code.

### 4. Create a Python wasi test with pytest

Create a pytest test in *python/itkwasm-median-wasi/test/*.

## Enjoy ITK!